In [1]:
from tqdm import tqdm

import numpy as np
import pandas as pd
import pandas_bokeh
import matplotlib.pyplot as plt
import seaborn as sns
import time, math, os, glob, wget, shutil, joblib, warnings, sys

time.clock = time.time
pandas_bokeh.output_notebook()

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

%load_ext autotime

Loading BokehJS ...

time: 98 µs (started: 2024-01-17 19:23:45 -08:00)


In [17]:
filepath = os.path.join('aGVsbG8udHh0.ENCR')
filesize = os.path.getsize(filepath)
print("File size: %s bytes" % filesize)

File size: 4224 bytes
time: 9.61 ms (started: 2024-01-17 19:25:34 -08:00)


In [18]:
num_blocks = math.ceil(filesize / 4096)
num_blocks

2

time: 2.29 ms (started: 2024-01-17 19:25:34 -08:00)


In [19]:
def read_bytearray(filepath):
    with open(filepath, mode='rb') as file:
        byte_data = file.read()
        return np.array(list(byte_data))

time: 629 µs (started: 2024-01-17 19:25:35 -08:00)


In [20]:
byte_array = read_bytearray(filepath)
assert(len(byte_array) == filesize)
byte_array[0:10]

array([ 37, 154,  92, 218, 214,  92,  47,   7,  95, 236])

time: 4.17 ms (started: 2024-01-17 19:25:38 -08:00)


In [21]:
def array_split_given_size(a, size):
    return np.split(a, np.arange(size,len(a),size))

time: 427 µs (started: 2024-01-17 19:25:39 -08:00)


In [22]:
block_datas = array_split_given_size(byte_array, 4096)
assert(len(block_datas) == num_blocks)
#assert(len(block_datas[0]) == 4096)
block_datas[0]

array([ 37, 154,  92, ..., 187,  78, 123])

time: 2.83 ms (started: 2024-01-17 19:25:40 -08:00)


In [23]:
from collections import Counter
eps = 1e-10

def calculate_histogram(block_data):
    counter = Counter(block_data)
    bins = np.arange(0, 256)
    counts = [counter[i] for i in bins]
    assert(bins[0] == 0)
    assert(bins[-1] == 255)
    df = pd.DataFrame({
        'bin': bins,
        'count': counts,
    })
    df['prob'] = df['count'] / np.sum(counts)
    df['prob'] = [np.max([eps, x]) for x in df['prob']]
    return df

def plot_histogram(df, title='Histogram', show_y='count'):
    pbar = df.plot_bokeh.bar(
        ylabel=show_y,
        title=title,
        x='bin',
        y=show_y,
        alpha=0.9,
        figsize=(1000,500),
        xticks = [0, 50, 100, 150, 200, 255]
    )
    return pbar

time: 1.23 ms (started: 2024-01-17 19:25:41 -08:00)


In [24]:
first_block = block_datas[0]
df = calculate_histogram(first_block)
display(df.head())

,bin,count,prob
0,0,17,0.004150
1,1,13,0.003174
2,2,13,0.003174
3,3,17,0.004150
4,4,14,0.003418


time: 16.8 ms (started: 2024-01-17 19:25:41 -08:00)


In [25]:
def calculate_shannon_entropy(probs):
    return -np.sum(probs * np.log2(probs))

time: 353 µs (started: 2024-01-17 19:25:42 -08:00)


In [26]:
shannon_entropy = calculate_shannon_entropy(df['prob'].tolist())
print("The Shannon entropy of first block is: %s" % shannon_entropy)

The Shannon entropy of first block is: 7.954636020859365
time: 769 µs (started: 2024-01-17 19:25:43 -08:00)


In [27]:
dfs = [calculate_histogram(block_data) for block_data in block_datas]
entropies = [calculate_shannon_entropy(df['prob'].tolist()) for df in dfs]
entropies

[7.954636020859365, 6.507660290422032]

time: 10.7 ms (started: 2024-01-17 19:25:44 -08:00)


In [28]:
np.mean(entropies)

7.231148155640698

time: 2.79 ms (started: 2024-01-17 19:25:46 -08:00)


## File Encryption

In [14]:
from Crypto.Cipher import AES
import os

def encrypt_file(file_path, key, desired_ext='.enc'):
    # Generate a random initialization vector (IV)
    iv = os.urandom(16)

    # Create AES cipher object with the provided key and mode
    cipher = AES.new(key, AES.MODE_CBC, iv)

    # Read the contents of the file
    with open(file_path, 'rb') as file:
        plaintext = file.read()

    # Pad the plaintext to match the block size of AES (16 bytes)
    padded_plaintext = pad(plaintext)

    # Encrypt the padded plaintext
    ciphertext = cipher.encrypt(padded_plaintext)

    # Write the IV and ciphertext to a new file
    encrypted_file_path = file_path + desired_ext
    with open(encrypted_file_path, 'wb') as file:
        file.write(iv + ciphertext)

    print("File encrypted successfully.")

def pad(data):
    # Calculate the number of bytes to pad
    padding_len = AES.block_size - (len(data) % AES.block_size)

    # Pad the data with the number of bytes to pad
    padding = bytes([padding_len]) * padding_len

    return data + padding

time: 249 ms (started: 2024-01-17 19:23:51 -08:00)


In [16]:
key = b'Sixteen byte key'  # Replace with your desired encryption key
# file saved 
encrypt_file(filepath, key, desired_ext='.enc')

SystemError: PY_SSIZE_T_CLEAN macro must be defined for '#' formats

time: 408 ms (started: 2024-01-17 19:24:29 -08:00)
